In [ ]:
import os
import gc
import cv2
import time
import random
import glob
from PIL import Image
import  matplotlib.pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [ ]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        
    def forward(self, image):
        x = self.model(image)
        return x
    
    
def get_test_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img = cv2.imread(data['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img/255, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

# CFG

In [ ]:
CFG= {
    'fold': 4,
    'img_size': 128,
    'TTA': False,
#     'model': ['./train_model/cv0_best.pth'],
    'model': [
        './test_model/convnextv2_tiny_imgsz128_PL3/cv4_best.pth',
        './test_model/effv2s_imgsz128_PL3/cv4_best.pth',
    ]
}
CFG['model']= [ torch.load(m, map_location= 'cuda:0') for m in CFG['model']]
print(f"length of model: {len(CFG['model'])}")

# Prepare Dataset

In [ ]:
df= pd.read_csv('Data/train.csv')
train_df= df[df['fold']!=CFG['fold']].reset_index(drop=True)
valid_df= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_df)}')
print(f'valid dataset: {len(valid_df)}')

valid_dataset= Customize_Dataset(valid_df.iloc[:], get_test_transform(CFG['img_size']))
valid_loader= DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=0)
valid_df.head()

In [ ]:
def inference(model, img):
    
    img= torch.unsqueeze(img, 0).cuda()
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            if CFG['TTA']:
                imgs= torch.cat([
                            img, 
                            img.flip(-1), 
                            img.flip(-2), 
                            img.flip(-1).flip(-2)
                        ], dim=0)
                
                ## tensor_trt can't use bs!=1
                for j in range(CFG['TTA']):
                    p= m(imgs[j:j+1])
                    if j==0: ps= p
                    else: ps+= p
                pred= ps/CFG['TTA']
            else:
                pred= m(img)[0]
                
        if i==0: preds= pred.softmax(dim=-1)
        else: preds+= pred.softmax(dim=-1)
            
    pred= preds/len(model)
    pred= pred.cpu().numpy()
    return pred

In [ ]:
valid_df['pred_cls']= None
valid_df['pred_prob']= None
count= 0
for i, data in enumerate(tqdm(valid_loader)):
    for j in range(len(data['image'])):
        img= data['image'][j]
#         print(img.shape)
        pred= inference(CFG['model'], img)
        valid_df.loc[count, 'pred_cls']= pred.argmax(0)
        valid_df.at[count, 'pred_prob']= pred.tolist()
        count+= 1
valid_df.head()

In [ ]:
valid_df['pred_cls'].value_counts()

In [ ]:
valid_df['pred_cls'].hist()

# Make Pseudo Label

# Confusion_Matrix

In [ ]:
from metrics import *

label= valid_df['label'].values
pred= valid_df['pred_prob'].values
pred= np.array([np.array(p) for p in pred])

recall= Mean_Recall(pred, label)
print(f'mean_recall: {recall}')

cmap= padded_cmap(pred, label)
print(f'cmap: {cmap}')

# show error img

# Grad_Cam